In [ ]:
%%capture output

from datetime import datetime, timedelta
from SQLiteFun import execution, executionALL
from hapiclient import hapi
from time import sleep

# check and see if data can be retrieved from HAPI server
def DataChecker(server, dataset, start, stop, parameters, attempts):
    # control loop
    dataFound = False
    
    # place in a try/except to wait x seconds and try again if this fails
    try:
        # Get data
        #sleep(5.0)
        data, meta = hapi(server, dataset, parameters, start, stop)

    # if error arises
    # needs to be specific error, not blank
    except:
        print("Error raised, trying again")

    # if no error arises
    else:
        dataLen = len(data)
        print("The length of data is " + str(dataLen))

        # search for data
        while not dataFound:
            for entry in data:
                #print(type(data[0][1]))
                
                # possible type mismatch when using boolean operator

                # once data is found, end loop
                if str(data[0][1]) != "":
                    dataFound = True
                    print("Data was successfully accessed")
                    print("Example data looks like " + str(data[0][1]))
                    break
    finally:
        attempts += 1
        return dataFound, attempts

# retrieve prodKeys associated with HAPI URLs

# fails data checks 
#HapiStmt = """SELECT prodKey FROM MetadataEntries WHERE url LIKE "%/hapi" LIMIT 1 OFFSET 9"""
#HapiStmt = """SELECT prodKey FROM MetadataEntries WHERE url LIKE "%/hapi" LIMIT 1 OFFSET 10"""
#HapiStmt = """SELECT prodKey FROM MetadataEntries WHERE url LIKE "%/hapi" LIMIT 2 OFFSET 15"""
#HapiStmt = """SELECT prodKey FROM MetadataEntries WHERE url LIKE "%/hapi" LIMIT 2 OFFSET 24"""

HapiStmt = """ SELECT prodKey FROM MetadataEntries WHERE url LIKE "%/hapi" """
prodKeys = []
prodKeys = execution(HapiStmt)
#prodKeys = ['HELIOS1_E6_1HOUR_PARTICLE_FLUX']
#print("The prodKeys are " + str(prodKeys))


server = 'https://cdaweb.gsfc.nasa.gov/hapi'
#dataset    = 'HELIOS1_E6_1HOUR_PARTICLE_FLUX'
#start      = '2011-08-06T00:00:00'
#stop       = '2011-08-06T00:30:00'
#parameters = 'SE_LAT'
#data, meta = hapi(server, dataset, parameters, start, stop)
#print(data)

# iterate thru prodKeys
for prodKey in prodKeys:
    # check if multiple prodKeys for same URL (mult keys in one string)
    if ", " in prodKey:
        print("This HAPI URL has multiple product keys.")
        index = prodKeys.index(prodKey)
        prodKey = prodKey.replace("\'", "")
        # keep separating them until each prodKey is in own string
        # while ", " in prodKey:
        before, sep, after = prodKey.partition(", ")
        prodKeys[index] = before # remove this line if need to check all keys
        #prodKeys[index] = after
        #prodKeys.insert(index, before)
        prodKey = prodKeys[index]
    
    # control loop
    dataFound = False
    # list that holds all parameters for a given server
    paramNames =  []

    dataset = str(prodKey)

    # retrieve all parameters and the start and stop date from the server
    sleep(5.0)
    meta = hapi(server,dataset)
    # get parameters
    for k, v in meta.items():
        if k == "parameters":
            for params in v:
                for key, value in params.items():
                    if key == "name":
                        paramNames.append(value)
        # get start date and create a stop date by adding 30 min to start
        elif k == "startDate":
            start = v
            date, sep, time = start.partition("T")
            time = time.replace("Z", "")
            dt_string = date + " " + time
            dt_obj = datetime.strptime(dt_string, "%Y-%m-%d %H:%M:%S")
            stop = dt_obj + timedelta(minutes=30)
            stop = str(stop)
            stop = stop.replace(" ", "T") + "Z"

    #print("Start time is " + start)
    #print("Stop time is " + stop)
    #print(paramNames)
    attempts = 0

    # iterate thru parameters in a server to see if any data can be pulled
    for parameters in paramNames[1:]:
        if attempts < 3:
            if not dataFound:
                print("Checking parameter " + str(parameters) + " in HAPI record with id " + dataset)
                dataFound, attempts = DataChecker(server, dataset, start, stop, parameters, attempts)
        # if 3 fails in a row -> no data
        else:
        # inputs "HAPI info check passed after 1 attempt. HAPI data check failed after 3 attempts."
        #     into "Error" column in TestResults associated with that HAPI URL
            print("No data was found")
            HAPIErrorStmt = f""" UPDATE TestResults
                            SET Errors = 'HAPI info check passed after 1 attempt. HAPI data check \
                            failed after 3 attempts.'
                            FROM (SELECT SPASE_id, prodKey FROM TestResults
                                    INNER JOIN MetadataEntries USING (SPASE_id))
                            WHERE prodKey = '{dataset}' """
            Record_id = execution(f""" SELECT rowNum 
                                    FROM (SELECT TestResults.rowNum, SPASE_id, prodKey FROM TestResults 
                                        INNER JOIN MetadataEntries USING (SPASE_id))
                                    WHERE prodKey = '{dataset}';""")
            executionALL(HAPIErrorStmt)
            print(f"Sent error message to a TestResults entry with the row number {Record_id}")
            break

with open("../DatalinkCheckOutput.txt", "w") as file:
    file.write(output.stdout)

In [ ]:
print("The program is done!")

In [ ]:
# call .py file directly from notebook
#%run ./HAPICheck.py > ../DatalinkCheckOutput.txt